In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
key=load_dotenv('OPENAI_API_KEY')

In [4]:
from langchain.chat_models import ChatOpenAI
client=ChatOpenAI(openai_api_key=key,model_name='gpt-4o',temperature=0.9)

e:\GENAI\MCQ-GEN\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE1="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [43]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE1
)


In [44]:
chain1 = LLMChain(llm=client, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [46]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [47]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['quiz', 'subject'],
    template=TEMPLATE2
)


In [48]:
chain2 = LLMChain(llm=client, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)



In [49]:
FinalChain = SequentialChain(chains=[chain1, chain2], input_variables=['text', 'number', 'subject', 'tone', 'response_json'], output_variables=['review'], verbose=True)

In [14]:
filepath=r'E:\GENAI\MCQ-GEN\paragraph.txt'

In [15]:
filepath

'E:\\GENAI\\MCQ-GEN\\paragraph.txt'

In [16]:
with open(filepath,'r') as file:
    data=file.read()

In [17]:
data

"Coronavirus disease 2019 (COVID-19) is a contagious disease caused by the coronavirus SARS-CoV-2. The first known case was identified in Wuhan, China, in December 2019.[7] Most scientists believe the SARS-CoV-2 virus entered into human populations through natural zoonosis, similar to the SARS-CoV-1 and MERS-CoV outbreaks, and consistent with other pandemics in human history.[8][9] Social and environmental factors including climate change, natural ecosystem destruction and wildlife trade increased the likelihood of such zoonotic spillover.[10][11][12][13] The disease quickly spread worldwide, resulting in the COVID-19 pandemic.\n\nThe symptoms of COVIDâ€‘19 are variable but often include fever,[14] fatigue, cough, breathing difficulties, loss of smell, and loss of taste.[15][16][17] Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who are infected do not develop noticeable symptoms.[18][19] Of those who develop symptoms noticeable enough t

In [18]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [52]:
NUMBER = 5 
SUBJECT = "covid"
TONE = "simple"


In [53]:
# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = FinalChain(
        {
            "text": data,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Sample text for generating the quiz.
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 5 multiple choice questions for covid students in simple tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 5 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "c

In [54]:
print(f"TotalCost:{cb.total_cost}")
print(f"TotalToken:{cb.total_tokens}")
print(f"InputTokens{cb.prompt_tokens}")
print(f"OutputTokens{cb.completion_tokens}")

TotalCost:0.015965
TotalToken:1549
InputTokens727
OutputTokens822


In [55]:
response

{'text': 'Sample text for generating the quiz.',
 'number': 5,
 'subject': 'covid',
 'tone': 'simple',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'review': '**Complexity Analysis:**\nThe quiz questions are straightforward and pertain more to quiz creation and administration than to evaluating students\' understanding of COVID-19. The complexity seems low for students familiar with course content.\n\n**Updated Quiz Questions:**\nHere is a revised quiz, aligned with the cognitive and analytical abilities of the students focu

In [56]:
res=response.get("review")

In [57]:
print(res)

**Complexity Analysis:**
The quiz questions are straightforward and pertain more to quiz creation and administration than to evaluating students' understanding of COVID-19. The complexity seems low for students familiar with course content.

**Updated Quiz Questions:**
Here is a revised quiz, aligned with the cognitive and analytical abilities of the students focusing on actual COVID-19 content:

```json
{
  "1": {
    "mcq": "What is COVID-19?",
    "options": {
      "a": "A type of food",
      "b": "A viral infection",
      "c": "A computer program",
      "d": "A book"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "Which of the following is a common symptom of COVID-19?",
    "options": {
      "a": "Headache",
      "b": "Cough",
      "c": "Blurry vision",
      "d": "Toothache"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "How can you protect yourself from COVID-19?",
    "options": {
      "a": "Washing hands frequently",
      "b": "Eating more sugar",
      "c": 

In [62]:
# Extract the JSON part from the response
try:
    start = response['review'].find("```json") + len("```json")
    end = response['review'].find("```", start)
    json_str = response['review'][start:end].strip()

    # Print the extracted JSON string
    print("Extracted JSON string:", json_str)

    # Parse the JSON string
    quiz = json.loads(json_str)
    print("Parsed JSON:", quiz)



except Exception as e:
    print("Error:", e)
    print("Traceback:")
    traceback.print_exc()
    print("Response content:")
    print(response)

Extracted JSON string: {
  "1": {
    "mcq": "What is COVID-19?",
    "options": {
      "a": "A type of food",
      "b": "A viral infection",
      "c": "A computer program",
      "d": "A book"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "Which of the following is a common symptom of COVID-19?",
    "options": {
      "a": "Headache",
      "b": "Cough",
      "c": "Blurry vision",
      "d": "Toothache"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "How can you protect yourself from COVID-19?",
    "options": {
      "a": "Washing hands frequently",
      "b": "Eating more sugar",
      "c": "Avoiding exercise",
      "d": "Sleeping less"
    },
    "correct": "a"
  },
  "4": {
    "mcq": "What should you do if you feel unwell with COVID-19 symptoms?",
    "options": {
      "a": "Ignore it and continue as normal",
      "b": "Take medicine without consulting a doctor",
      "c": "Stay at home and call a healthcare provider",
      "d": "Go to a crowded place"
    },
 

In [64]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quiz_table_data

[{'MCQ': 'What is COVID-19?',
  'Choices': 'a: A type of food | b: A viral infection | c: A computer program | d: A book',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is a common symptom of COVID-19?',
  'Choices': 'a: Headache | b: Cough | c: Blurry vision | d: Toothache',
  'Correct': 'b'},
 {'MCQ': 'How can you protect yourself from COVID-19?',
  'Choices': 'a: Washing hands frequently | b: Eating more sugar | c: Avoiding exercise | d: Sleeping less',
  'Correct': 'a'},
 {'MCQ': 'What should you do if you feel unwell with COVID-19 symptoms?',
  'Choices': 'a: Ignore it and continue as normal | b: Take medicine without consulting a doctor | c: Stay at home and call a healthcare provider | d: Go to a crowded place',
  'Correct': 'c'},
 {'MCQ': 'What is a key measure to prevent the spread of COVID-19?',
  'Choices': 'a: Wearing masks | b: Sharing personal items | c: Eating out frequently | d: Ignoring health guidelines',
  'Correct': 'a'}]

In [65]:
# Convert to DataFrame for easy viewing
df = pd.DataFrame(quiz_table_data)
print(df)

                                                 MCQ  \
0                                  What is COVID-19?   
1  Which of the following is a common symptom of ...   
2        How can you protect yourself from COVID-19?   
3  What should you do if you feel unwell with COV...   
4  What is a key measure to prevent the spread of...   

                                             Choices Correct  
0  a: A type of food | b: A viral infection | c: ...       b  
1  a: Headache | b: Cough | c: Blurry vision | d:...       b  
2  a: Washing hands frequently | b: Eating more s...       a  
3  a: Ignore it and continue as normal | b: Take ...       c  
4  a: Wearing masks | b: Sharing personal items |...       a  


In [ ]:
quiz=pd.DataFrame(quiz_table_data)


In [ ]:
quiz

""


In [ ]:
quiz.to_csv("Covid19.csv",index=False)